In [1]:
from buildstock_query import BuildStockQuery
import pandas as pd

In [2]:
my_run = BuildStockQuery(db_name='euss-tests',
                        table_name='res_test_03_2018_10k_20220607',
                        workgroup='eulp',
                        buildstock_type='resstock')

INFO:buildstock_query.query_core:Loading res_test_03_2018_10k_20220607 ...
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:buildstock_query.query_core:84 queries cache read from res_test_03_2018_10k_20220607_query_cache.pkl.
INFO:buildstock_query.query_core:84 queries cache is updated.
INFO:buildstock_query.query_core:84 queries cache read from res_test_03_2018_10k_20220607_query_cache.pkl.
INFO:buildstock_query.query_core:Cache already upto date.
INFO:buildstock_query.main:Getting Success counts...
INFO:buildstock_query.report_query:Checking integrity with ts_tables ...


         Fail  Unapplicaple  Success      Sum  Applied %  no-chng  bad-chng  \
upgrade                                                                       
0          11             0     9989  10000.0        0.0        0         0   
1           0           223     9766   9989.0       97.8       48         6   
2           0           155     9834   9989.0       98.4       54         7   
3           0             6     9983   9989.0       99.9       12       235   
4           0             6     9983   9989.0       99.9        0       169   
5           0            53     9936   9989.0       99.5       12       742   
6           0            55     9934   9989.0       99.4        0        34   
7           0          2493     7496   9989.0       75.0       42        35   
8           0             0     9989   9989.0      100.0        0        37   
9           0             0     9989   9989.0      100.0        0        34   
10          0             0     9989   9989.0      1

INFO:buildstock_query.query_core:84 queries cache saved to res_test_03_2018_10k_20220607_query_cache.pkl


In [4]:
baseline_agg = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                       group_by=['geometry_building_type_recs'])
baseline_agg


,geometry_building_type_recs,sample_count,units_count,fuel_use_electricity_total_m_btu
0,Single-Family Detached,7121,9.483182e+07,4.413546e+09
1,Mobile Home,703,9.361996e+06,4.506122e+08
2,Multi-Family with 2 - 4 Units,446,5.939474e+06,1.385683e+08
3,Multi-Family with 5+ Units,1189,1.583416e+07,4.099084e+08
4,Single-Family Attached,530,7.058119e+06,2.094629e+08


In [5]:
ts_agg = my_run.agg.aggregate_timeseries(enduses=['fuel_use__electricity__total__kwh'],
                                         group_by=['geometry_building_type_recs'])
ts_agg

INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.


,geometry_building_type_recs,time,sample_count,units_count,fuel_use__electricity__total__kwh
0,Mobile Home,2018-01-02 01:15:00,703,9.361996e+06,1.140340e+07
1,Mobile Home,2018-01-05 03:15:00,703,9.361996e+06,1.004237e+07
2,Mobile Home,2018-01-05 13:45:00,703,9.361996e+06,5.873500e+06
3,Mobile Home,2018-01-07 09:45:00,703,9.361996e+06,9.015935e+06
4,Mobile Home,2018-01-09 13:15:00,703,9.361996e+06,3.875666e+06
...,...,...,...,...,...
175195,Multi-Family with 5+ Units,2018-06-30 14:00:00,1189,1.583416e+07,4.785724e+06
175196,Multi-Family with 5+ Units,2018-07-02 21:15:00,1189,1.583416e+07,4.119971e+06
175197,Multi-Family with 5+ Units,2018-07-04 18:00:00,1189,1.583416e+07,4.948234e+06
175198,Multi-Family with 5+ Units,2018-07-10 09:00:00,1189,1.583416e+07,3.577294e+06


In [6]:
my_run.savings.savings_shape(upgrade_id=2, enduses=['fuel_use__electricity__total__kwh'],
                             annual_only=False)

,time,sample_count,units_count,fuel_use__electricity__total__kwh__baseline,fuel_use__electricity__total__kwh__savings
0,2018-01-01 00:15:00,9989,1.330256e+08,7.148833e+07,1.966400e+07
1,2018-01-01 00:30:00,9989,1.330256e+08,7.227770e+07,1.960490e+07
2,2018-01-01 00:45:00,9989,1.330256e+08,7.189331e+07,1.947231e+07
3,2018-01-01 01:00:00,9989,1.330256e+08,7.202095e+07,1.937339e+07
4,2018-01-01 01:15:00,9989,1.330256e+08,7.083665e+07,1.915390e+07
...,...,...,...,...,...
35035,2018-12-31 23:00:00,9989,1.330256e+08,4.153427e+07,5.265636e+06
35036,2018-12-31 23:15:00,9989,1.330256e+08,3.732818e+07,5.140162e+06
35037,2018-12-31 23:30:00,9989,1.330256e+08,3.620637e+07,5.337922e+06
35038,2018-12-31 23:45:00,9989,1.330256e+08,3.537120e+07,5.329013e+06


In [7]:
my_run.savings.savings_shape(upgrade_id=2, enduses=['fuel_use__electricity__total__kwh'],
                             group_by=['geometry_building_type_recs'],
                             annual_only=False)

,time,sample_count,units_count,fuel_use__electricity__total__kwh__baseline,fuel_use__electricity__total__kwh__savings,geometry_building_type_recs
0,2018-01-01 00:15:00,703,9.361996e+06,1.098386e+07,2.682538e+06,Mobile Home
1,2018-01-01 00:30:00,703,9.361996e+06,1.093122e+07,2.634476e+06,Mobile Home
2,2018-01-01 00:45:00,703,9.361996e+06,1.077442e+07,2.593965e+06,Mobile Home
3,2018-01-01 01:00:00,703,9.361996e+06,1.080007e+07,2.552429e+06,Mobile Home
4,2018-01-01 01:15:00,703,9.361996e+06,1.059866e+07,2.516619e+06,Mobile Home
...,...,...,...,...,...,...
175195,2018-12-31 23:00:00,7121,9.483182e+07,3.171454e+07,4.143382e+06,Single-Family Detached
175196,2018-12-31 23:15:00,7121,9.483182e+07,2.811727e+07,3.993204e+06,Single-Family Detached
175197,2018-12-31 23:30:00,7121,9.483182e+07,2.729993e+07,4.172387e+06,Single-Family Detached
175198,2018-12-31 23:45:00,7121,9.483182e+07,2.672484e+07,4.160202e+06,Single-Family Detached


In [8]:
my_run.savings.savings_shape(upgrade_id=2, enduses=['fuel_use__electricity__total__kwh'],
                             group_by=['geometry_building_type_recs'],
                             get_quartiles=True,
                             annual_only=False)

,time,sample_count,units_count,fuel_use__electricity__total__kwh__baseline,fuel_use__electricity__total__kwh__savings,fuel_use__electricity__total__kwh__savings__quartiles,geometry_building_type_recs
0,2018-01-01 00:15:00,703,9.361996e+06,1.098386e+07,2.682538e+06,"[-0.05399999999999999, 0.0, 0.0159999999999999...",Mobile Home
1,2018-01-01 00:30:00,703,9.361996e+06,1.093122e+07,2.634476e+06,"[-0.031, 0.0, 0.01599999999999996, 0.169000000...",Mobile Home
2,2018-01-01 00:45:00,703,9.361996e+06,1.077442e+07,2.593965e+06,"[-0.03300000000000003, 0.0, 0.0150000000000000...",Mobile Home
3,2018-01-01 01:00:00,703,9.361996e+06,1.080007e+07,2.552429e+06,"[-0.027999999999999997, 0.0, 0.016, 0.16399999...",Mobile Home
4,2018-01-01 01:15:00,703,9.361996e+06,1.059866e+07,2.516619e+06,"[-0.024999999999999994, 0.0, 0.016, 0.15799999...",Mobile Home
...,...,...,...,...,...,...,...
175195,2018-12-31 23:00:00,7121,9.483182e+07,3.171454e+07,4.143382e+06,"[-0.473, -0.020999999999999963, 0.0, 0.0040000...",Single-Family Detached
175196,2018-12-31 23:15:00,7121,9.483182e+07,2.811727e+07,3.993204e+06,"[-0.489, -0.020999999999999963, 0.0, 0.0040000...",Single-Family Detached
175197,2018-12-31 23:30:00,7121,9.483182e+07,2.729993e+07,4.172387e+06,"[-0.518, -0.017, 0.0, 0.0040000000000000036, 0...",Single-Family Detached
175198,2018-12-31 23:45:00,7121,9.483182e+07,2.672484e+07,4.160202e+06,"[-0.587, -0.01799999999999996, 0.0, 0.00400000...",Single-Family Detached
